# 제 4 장 데이터 수집과 정리
___

## __사전설정__
---

(1) 저장소 데이터 가져오기

In [ ]:
!rm -rf /content/BizDataAnalysis/

In [ ]:
!git clone https://github.com/BizStat/BizDataAnalysis.git


(2) matplotlib 환경에서 한글 사용

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -f -v
!rm ~/.cache/matplotlib -rf

런타임 메뉴에서 '세션 다시 시작' 후 다음의 명령문 실행

In [ ]:
from matplotlib import rc
rc('font', family='NanumMyeongjo')
rc('axes', unicode_minus=False)

___

## 4.2 __Open API 이용__

In [ ]:
import requests as req
import json
import pandas as pd

(1) 한국은행 Open API

* 홈페이지 : https://ecos.bok.or.kr/api/#/

* Open API 인증키 신청 후 [My Page]에서 확인

In [ ]:
BOK_KEY = 'N1VG2759HK9V3HDMUG1U'

* 이용 예시

In [ ]:
uri = 'http://ecos.bok.or.kr/api/StatisticTableList/N1VG2759HK9V3HDMUG1U/json/kr/1/10/102Y004'
resp = req.get(uri)

In [ ]:
resp.text

In [ ]:
res1 = json.loads(resp.text)

In [ ]:
res1

In [ ]:
res1['StatisticTableList']

In [ ]:
res1['StatisticTableList']['row']

In [ ]:
df1 = pd.json_normalize(res1['StatisticTableList']['row'])

In [ ]:
df1

* 통계 용어 사전 활용

In [ ]:
uri = 'http://ecos.bok.or.kr/api/StatisticWord/N1VG2759HK9V3HDMUG1U/json/kr/1/10/'

In [ ]:
word = '소비자동향지수'

In [ ]:
resp = req.get(uri+word)

In [ ]:
resp.status_code

In [ ]:
resp.text

In [ ]:
res2 = json.loads(resp.text)

In [ ]:
df2 = pd.json_normalize(res2['StatisticWord']['row'])

In [ ]:
df2

* 최근 3년간의 소비자물가지수(분기별 총지수) 가져오기

In [ ]:
uri_fmt = 'http://ecos.bok.or.kr/api/StatisticSearch/{}/json/kr/1/1000/{}/{}/{}/{}/{}'

In [ ]:
통계코드,작성주기,시작시점,종료시점,항목코드= '901Y009','Q','2021Q1','2023Q4','0'

In [ ]:
uri = uri_fmt.format(BOK_KEY,통계코드,작성주기,시작시점,종료시점,항목코드)

In [ ]:
resp = req.get(uri)

In [ ]:
resp

In [ ]:
tmp = json.loads(resp.text)

In [ ]:
res = pd.json_normalize(tmp['StatisticSearch']['row'])

In [ ]:
res

(2) 네이버 검색 API

* 애플리케이션 등록 후 API 인증 정보

In [ ]:
CLT_ID = 'Q8Pw7CUP8kMmHB51FSGA'
CLT_SCR = 'wguPXntry8'

* 뉴스 검색

In [ ]:
API_URL = 'https://openapi.naver.com/v1/search/news.json'
params = {
    'query' : '방탄소년단',
    'display' : 100,
    'start' : 1,
    'sort' : 'date'
}
headers = {
    'X-Naver-Client-Id' : CLT_ID,
    'X-Naver-Client-Secret' : CLT_SCR
}

In [ ]:
resp = req.get(API_URL, params=params, headers=headers)

In [ ]:
resp.status_code

In [ ]:
cnt = resp.json()

In [ ]:
cnt['items']

In [ ]:
df1 = pd.json_normalize(cnt['items'])

In [ ]:
df1 = pd.json_normalize((resp.json())['items'])

In [ ]:
df1.head()

In [ ]:
df1.dtypes

(3) kakao Karlo API

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# REST API 호출, 이미지 파일 처리에 필요한 라이브러리
import requests
import json
import urllib
from PIL import Image

# [내 애플리케이션] > [앱 키] 에서 확인한 REST API 키 값 입력
REST_API_KEY = '0721df54079135c7625d18e86cf1ee12'

# 이미지 생성하기 요청
def t2i(prompt, negative_prompt):
    r = requests.post(
        'https://api.kakaobrain.com/v2/inference/karlo/t2i',
        json = {
            "version": "v2.1",
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "height": 1024,
            "width": 1024
        },
        headers = {
            'Authorization': f'KakaoAK {REST_API_KEY}',
            'Content-Type': 'application/json'
        }
    )
    # 응답 JSON 형식으로 변환
    response = json.loads(r.content)
    return response

# 프롬프트에 사용할 제시어
prompt = "A photo of a cute tiny tiger on the beach, daylight."
negative_prompt = "low quality, low contrast, draft, amateur, cut off, cropped, frame"

# 이미지 생성하기 REST API 호출
response = t2i(prompt, negative_prompt)

# 응답의 첫 번째 이미지 생성 결과 출력하기
Image.open(urllib.request.urlopen(response.get("images")[0].get("image")))

In [ ]:
# REST API 호출, 이미지 파일 처리에 필요한 라이브러리
import requests
import json
import io
import base64
import urllib
from PIL import Image

# [내 애플리케이션] > [앱 키] 에서 확인한 REST API 키 값 입력
REST_API_KEY = '0721df54079135c7625d18e86cf1ee12'

# 이미지 변환하기
def variations(image):
    r = requests.post(
        'https://api.kakaobrain.com/v2/inference/karlo/variations',
        json = {
            'version': 'v2.1',
            'image': image,
            'height': 1024,
            'width': 1024
        },
        headers = {
            'Authorization': f'KakaoAK {REST_API_KEY}',
            'Content-Type': 'application/json'
        }
    )
    # 응답 JSON 형식으로 변환
    response = json.loads(r.content)
    return response

# Base64 인코딩
def imageToString(img):
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='JPEG')
    my_encoded_img = base64.encodebytes(img_byte_arr.getvalue()).decode('ascii')
    return my_encoded_img

# 이미지 파일 불러오기
img = Image.open('/content/BizDataAnalysis/DATA/눈물의 여왕.jpg')

# 이미지를 Base64 인코딩하기
img_base64 = imageToString(img)

# 이미지 변환하기 REST API 호출
response = variations(img_base64)
response

# 응답의 첫 번째 이미지 생성 결과 출력하기
result = Image.open(urllib.request.urlopen(response.get("images")[0].get("image")))
result

In [ ]:
response

In [ ]:
# REST API 호출, 이미지 파일 처리에 필요한 라이브러리
import requests
import json
import io
import base64
import urllib
from PIL import Image

# [내 애플리케이션] > [앱 키] 에서 확인한 REST API 키 값 입력
REST_API_KEY = '0721df54079135c7625d18e86cf1ee12'

# 이미지 변환하기
def inpainting(image, mask):
    r = requests.post(
        'https://api.kakaobrain.com/v2/inference/karlo/inpainting',
        json = {
            'image': image,
            'mask': mask
        },
        headers = {
            'Authorization': f'KakaoAK {REST_API_KEY}',
            'Content-Type': 'application/json'
        }
    )
    # 응답 JSON 형식으로 변환
    response = json.loads(r.content)
    return response

# Base64 인코딩
def imageToString(img):
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='JPEG')
    my_encoded_img = base64.encodebytes(img_byte_arr.getvalue()).decode('ascii')
    return my_encoded_img

# 이미지 파일 불러오기
img = Image.open('/content/BizDataAnalysis/DATA/눈물의 여왕.jpg')
mask = Image.open('/content/BizDataAnalysis/DATA/눈물의 여왕.jpg')

# 이미지를 Base64 인코딩하기
img_base64 = imageToString(img)
mask_base64 = imageToString(mask)

# 이미지 변환하기 REST API 호출
response = inpainting(img_base64,mask_base64)
print(response)

# 응답의 첫 번째 이미지 생성 결과 출력하기
result = Image.open(urllib.request.urlopen(response.get("images")[0].get("image")))
result